Create distance dataset from a GPX track of a boat and one or more stationary hydrophones

In [ ]:
# Install depedencies
!pip install gpxpy==1.5.0 geopy==2.2.0 pandas==1.4.1 pydub==0.25.1

In [ ]:
from pathlib import Path
import gpxpy
from geopy.distance import distance
import pandas as pd
from pydub import AudioSegment

In [ ]:
gpx_path = "example.gpx"
NoBoatSection = -300000
hydrophones = RuntimeError("Replace me with proper hydrophone locations"); raise hydrophones

In [ ]:
with open("example.gpx", "r") as fd:
    gpx = gpxpy.parse(fd)

In [ ]:
def iterate_though_points():
    for route in gpx.routes:
        for point in route.points:
            yield {k: distance(v, (point.latitude, point.longitude)).m for k, v in hydrophones.items()}

In [ ]:
df = pd.DataFrame(iterate_though_points())

In [ ]:
def labeller(ele):
    if ele < 250:
        return "Close"
    else:
        return "Far"

df = df.applymap(labeller)

In [ ]:
none_path = Path.cwd() / "None" 
close_path = Path.cwd() / "Close"
far_path = Path.cwd() / "Far" 
none_path.mkdir(exist_ok=True)
close_path.mkdir(exist_ok=True)
far_path.mkdir(exist_ok=True)

for unit in hydrophones.keys():
    wav = AudioSegment.from_wav(f"{unit}.WAV")
    wav[NoBoatSection:].export(none_path / f"{unit}.wav", format="wav")
    sample_diff = (len(wav) - NoBoatSection) // len(df)
    window_size = sample_diff // 2
    for i, sample in enumerate(df[unit], 1):
        left_edge = i * sample_diff - window_size
        right_edge = i * sample_diff + window_size
        filename = Path.cwd() / sample / f"{unit}-{i * sample_diff}.wav"
        wav[left_edge:right_edge].export(filename, format="wav")